In [ ]:
import numpy as np
import pandas as pd
import os
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.sql.functions import col, count, mean, sum, avg, stddev, min, max, lit
from pyspark.ml.stat import Summarizer
from pyspark.ml.classification import LogisticRegression

In [ ]:
path_to_data = os.path.join(os.getcwd(),'data')
path_to_data

In [ ]:
spark.sql("use real_world_data_ed_omop_aug_2024")
spark.sql("show tables").toPandas()

In [ ]:
spark.catalog.refreshTable("delivery_elig_init_smm")
delivery_elig_init_smm = spark.sql('''
    select * from delivery_elig_init_smm
''').cache()
delivery_elig_init_smm.first()

In [ ]:
# one hot encoding
def ohe_with_map(
    df,              # spark dataframe
    cat_cols         # list of categorical columns for ohe
):   
    # Index the categorical columns
    indexers = [StringIndexer(inputCol=col, outputCol=col+"_index", stringOrderType="frequencyAsc").fit(df) for col in cat_cols]
    
    # Apply the indexers to the DataFrame and collect the encoding map
    index_maps = {}
    for indexer in indexers:
        df = indexer.transform(df)
        index_maps[indexer.getInputCol()] = dict(enumerate(indexer.labels))
    
    # One-hot encode the indexed columns
    encoders = [OneHotEncoder(inputCol=col + "_index", outputCol=col + "_ohe") for col in cat_cols]
    for encoder in encoders:
        df = encoder.transform(df)
    
    return df, index_maps 

In [ ]:
delivery_elig_init_smm_ohe = ohe_with_map(
    delivery_elig_init_smm,
    cat_cols = [
        "race_source_value",
        "ethnicity_source_value",
        "agegrp_at_event",
        "bed_size",
        "segment",
        "speciality",
        "zip_code",
        "delivery_type"
    ]
)
delivery_elig_init_smm2 = delivery_elig_init_smm_ohe[0]
delivery_elig_init_smm2.first()

In [ ]:
def bootstrap_lasso_logistic(
    df, 
    cov_lst,
    label_col,
    sample_frac = 0.9,
    n_iterations = 10, 
    alpha = 0.05
):
    coefs = []
    
    for i in range(n_iterations):
        # Create a random sample with replacement
        subsampled_df = df.sample(withReplacement=False, fraction=sample_frac)
        
        # train logistic lasso model
        assembler = VectorAssembler(inputCols=cov_lst, outputCol="features")
        train_data = assembler.transform(subsampled_df)
        lasso_logistic = LogisticRegression(featuresCol="features", labelCol=label_col, elasticNetParam=1.0)
        model = lasso_logistic.fit(train_data)
        
        # collect coef
        coefs.append(model.coefficients)

    # Calculate the lower and upper percentiles for the confidence interval
    coef_meds = np.percentile(coefs, 50, axis=0)
    lower_bounds = np.percentile(coefs, 100 * alpha / 2, axis=0)
    upper_bounds = np.percentile(coefs, 100 * (1 - alpha / 2), axis=0)
    feature_name = assembler.getInputCols()
    
    return feature_name, coef_meds, lower_bounds, upper_bounds

In [ ]:
cov_lst = [
     'race_source_value'
    ,'ethnicity_source_value'
    ,'agegrp_at_event'
    ,'los7up_ind'
    ,'delivery_type'        
    ,'bed_size'
    ,'speciality'
    ,'segment'
    ,'zip_code'
]
ohe_map = delivery_elig_init_smm_ohe[1]
cov_lst_ext = []
for v in cov_lst:
    if v in ohe_map:
        cov_lst_ext.extend(
            [v + f"_{k}" for k in list(ohe_map[v].keys())[:-1]]
        )
    else:
        cov_lst_ext.append(v)

In [ ]:
# multiple lasso logistic regresion with basic var, smmany
feature_name, coef_meds, lower_bounds, upper_bounds = bootstrap_lasso_logistic(
    delivery_elig_init_smm2, 
    cov_lst = [
         'race_source_value_ohe'
        ,'ethnicity_source_value_ohe'
        ,'agegrp_at_event_ohe'
        ,'los'
        ,'delivery_type_ohe'        
        ,'bed_size_ohe'
        ,'speciality_ohe'
        ,'segment_ohe'
        ,'zip_code_ohe'
    ],
    label_col = 'SMMANY_ind',
    sample_frac = 0.9,
    n_iterations = 20, 
    alpha = 0.05
)

In [ ]:
res_mlasso = pd.DataFrame({
    'feature': cov_lst_ext, 
    'coef': coef_meds,
    'lower': lower_bounds,
    'upper': upper_bounds
})
# res_mlasso
res_mlasso.to_csv(os.path.join(path_to_data,'multilasso_smmany_filter_init.csv'), index=False)

In [ ]:
# multiple lasso logistic regresion with basic var, smmany with 90 percentile correction
feature_name, coef_meds, lower_bounds, upper_bounds = bootstrap_lasso_logistic(
    delivery_elig_init_smm2, 
    cov_lst = [
         'race_source_value_ohe'
        ,'ethnicity_source_value_ohe'
        ,'agegrp_at_event_ohe'
        ,'los'
        ,'delivery_type_ohe'        
        ,'bed_size_ohe'
        ,'speciality_ohe'
        ,'segment_ohe'
        ,'zip_code_ohe'
    ],
    label_col = 'SMMANY90PCRT_ind',
    sample_frac = 0.9,
    n_iterations = 20, 
    alpha = 0.05
)

In [ ]:
res_mlasso = pd.DataFrame({
    'feature': cov_lst_ext, 
    'coef': coef_meds,
    'lower': lower_bounds,
    'upper': upper_bounds
})
# res_mlasso
res_mlasso.to_csv(os.path.join(path_to_data,'multilasso_smmany90pct_filter_init.csv'), index=False)

In [ ]:
# multiple lasso logistic regresion with all var, smmany
feature_name, coef_meds, lower_bounds, upper_bounds = bootstrap_lasso_logistic(
    delivery_elig_init_smm2, 
    cov_lst = [
         'race_source_value_ohe'
        ,'ethnicity_source_value_ohe'
        ,'agegrp_at_event_ohe'
        ,'los'
        ,'delivery_type_ohe'        
        ,'bed_size_ohe'
        ,'speciality_ohe'
        ,'segment_ohe'
        ,'zip_code_ohe'
        ,'hist_HEP_ind'
        ,'hist_IHD_ind'
        ,'hist_AST_ind'
        ,'hist_LIV_ind'
        ,'hist_AFIB_ind'
        ,'hist_STR_ind'
        ,'hist_CKD_ind'
        ,'hist_COPD_ind'
        ,'hist_HTN_ind'
        ,'hist_HF_ind'
        ,'hist_AIDS_ind'
        ,'hist_PVD_ind'
        ,'hist_RA_ind'
        ,'hist_AD_ind'
        ,'hist_DM_ind'
        ,'hist_COVID_ind'
        ,'hist_SUB_ind'
        ,'hist_ALC_ind'
    ],
    label_col = 'SMMANY_ind',
    sample_frac = 0.9,
    n_iterations = 20, 
    alpha = 0.05
)

In [ ]:
res_mlasso = pd.DataFrame({
    'feature': cov_lst_ext, 
    'coef': coef_meds,
    'lower': lower_bounds,
    'upper': upper_bounds
})
# res_mlasso
res_mlasso.to_csv(os.path.join(path_to_data,'multilasso_smmany_filter_init2.csv'), index=False)

In [ ]:
# multiple lasso logistic regresion with all var, smmany with 90 percentile correction
feature_name, coef_meds, lower_bounds, upper_bounds = bootstrap_lasso_logistic(
    delivery_elig_init_smm2, 
    cov_lst = [
         'race_source_value_ohe'
        ,'ethnicity_source_value_ohe'
        ,'agegrp_at_event_ohe'
        ,'los'
        ,'delivery_type_ohe'        
        ,'bed_size_ohe'
        ,'speciality_ohe'
        ,'segment_ohe'
        ,'zip_code_ohe'
        ,'hist_HEP_ind'
        ,'hist_IHD_ind'
        ,'hist_AST_ind'
        ,'hist_LIV_ind'
        ,'hist_AFIB_ind'
        ,'hist_STR_ind'
        ,'hist_CKD_ind'
        ,'hist_COPD_ind'
        ,'hist_HTN_ind'
        ,'hist_HF_ind'
        ,'hist_AIDS_ind'
        ,'hist_PVD_ind'
        ,'hist_RA_ind'
        ,'hist_AD_ind'
        ,'hist_DM_ind'
        ,'hist_COVID_ind'
        ,'hist_SUB_ind'
        ,'hist_ALC_ind'
    ],
    label_col = 'SMMANY90PCT_ind',
    sample_frac = 0.9,
    n_iterations = 20, 
    alpha = 0.05
)

In [ ]:
res_mlasso = pd.DataFrame({
    'feature': cov_lst_ext, 
    'coef': coef_meds,
    'lower': lower_bounds,
    'upper': upper_bounds
})
# res_mlasso
res_mlasso.to_csv(os.path.join(path_to_data,'multilasso_smmany90pct_filter_init2.csv'), index=False)